# API TRANSPORTE DE BUENOS AIRES
### Alumno: Valentín Tafura

Api link:
- https://api-transporte.buenosaires.gob.ar/console

API Transporte provee información GTFS dinámica y estática subtes dentro de la CABA con una granularidad medida en minutos (3 minutos aprox.)

### API Request 

In [1]:
from config import client_id, client_secret
from functions import request_api

# Request a la API
data_json = request_api(client_id, client_secret)

Conexión exitosa a la API


### Visualización del JSON

In [2]:
# Json obtenido de la API
data_json

{'Header': {'timestamp': 1714769873},
 'Entity': [{'ID': 'LineaA_A11',
   'Linea': {'Trip_Id': 'A11',
    'Route_Id': 'LineaA',
    'Direction_ID': 1,
    'start_time': '15:43:00',
    'start_date': '20240503',
    'Estaciones': [{'stop_id': '1059N',
      'stop_name': 'San Pedrito',
      'arrival': {'time': 1714769993, 'delay': 120},
      'departure': {'time': 1714770017, 'delay': 144}},
     {'stop_id': '1060N',
      'stop_name': 'San José de Flores',
      'arrival': {'time': 1714770053, 'delay': 180},
      'departure': {'time': 1714770077, 'delay': 204}},
     {'stop_id': '1061N',
      'stop_name': 'Carabobo',
      'arrival': {'time': 1714769933, 'delay': 60},
      'departure': {'time': 1714769957, 'delay': 84}},
     {'stop_id': '1062N',
      'stop_name': 'Puán',
      'arrival': {'time': 1714769993, 'delay': 120},
      'departure': {'time': 1714770017, 'delay': 144}},
     {'stop_id': '1063N',
      'stop_name': 'Primera Junta',
      'arrival': {'time': 1714769993, 'del

### Normalización del JSON
Debido a que el JSON posee valores anidados es necesario normalizarlo hasta obtener una estructura de tabla (DataFrame). El Request tambien brinda un TimeStamp que sera incluido en la columna temporal de la base de datos.

In [3]:
from functions import normalize_json
    
# Normalido el json y obtengo el timestamp del request
data, timestamp = normalize_json(data_json)

### Head del DataFrame

In [4]:
# Muestro las primeras 5 filas del DataFrame normalizado
data.head() if data is not False else None

,timestamp_event,line_id,trip_id,route_id,direction_id,start_time,start_date,stop_id,stop_name,arrival_time,arrival_delay,departure_time,departure_delay
0,2024-05-03 20:57:53,LineaA_A11,A11,LineaA,1,15:43:00,2024-05-03,1059N,San Pedrito,2024-05-03 20:59:53,120,2024-05-03 21:00:17,144
1,2024-05-03 20:57:53,LineaA_A11,A11,LineaA,1,15:43:00,2024-05-03,1060N,San José de Flores,2024-05-03 21:00:53,180,2024-05-03 21:01:17,204
2,2024-05-03 20:57:53,LineaA_A11,A11,LineaA,1,15:43:00,2024-05-03,1061N,Carabobo,2024-05-03 20:58:53,60,2024-05-03 20:59:17,84
3,2024-05-03 20:57:53,LineaA_A11,A11,LineaA,1,15:43:00,2024-05-03,1062N,Puán,2024-05-03 20:59:53,120,2024-05-03 21:00:17,144
4,2024-05-03 20:57:53,LineaA_A11,A11,LineaA,1,15:43:00,2024-05-03,1063N,Primera Junta,2024-05-03 20:59:53,120,2024-05-03 21:00:17,144


### Conexión a la Base de Datos de Redshift

In [5]:
from functions import database_connection
from config import credentials

# Conexión a la base de datos
conn = database_connection(credentials)

Conexión exitosa a la base de datos


### Creación de la tabla 'subte' en la Base de Datos de Redshift

In [6]:
from functions import create_table

# Creación de la tabla 'subte' en la base de datos
create_table(conn, 'subte')

Tabla 'subte' creada


### Ingesta de datos obtenidos de la API en la tabla 'subte'

Se van a ingestar los datos en la tabla solo si no se han ingresado previamente en la base de datos. 
Se hace una consulta a la columna timestamp_event de la tabla y se verica si el TimeStamp obtenido en el API Request no se encuentra en la misma.

In [7]:
from functions import data_ingestion

# Ingesta de datos obtenidos desde la API a la tabla 'subte'
data_ingestion(conn, data, timestamp, 'subte')

Se ingresaron 136 registros en la tabla 'subte'


In [8]:
# Cierro la conexión a la base de datos
conn.close() if conn is not False else None